In [1]:
import torch
import torch.nn as nn
import json
from model import Seq2Seq
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from data_loader import get_loader

In [2]:
with open('./data/word2id.json', 'r') as f:
    word2id = json.load(f)
    
src_root = './data/task1/source/'
trg_root = './data/task1/target/'
data_loader = get_loader(src_root, trg_root, 10, num_workers=2, shuffle=False)

In [3]:
w2i = word2id
i2w = dict()
for k,v in w2i.items():
    i2w[v]=k

In [4]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [5]:
vocab_size = len(word2id)
model = Seq2Seq(vocab_size)
model.load_state_dict(torch.load('./data/model.pth'))
model.cuda()

log_step = 100
num_epochs = 10
lr = 0.001

for e in range(num_epochs):
    for i, (sources, targets, source_lengths, target_lengths) in enumerate(data_loader):
        
        # convert tensor to variable
        sources = to_var(sources)
        targets_in = to_var(targets[:, :-1])
        targets_out = to_var(targets[:, 1:])
        target_lengths = [x-1 for x in target_lengths] 
        
        # Forward pass
        h, c = model.encode(sources, source_lengths)
        output = model.decode(targets_in, target_lengths, h, c)
        
        # Compute loas
        targets_out = pack(targets_out, target_lengths, batch_first=True)[0]
        

NameError: name 'loss' is not defined

In [6]:
outputs = output.max(1)

In [14]:
(targets_out == outputs).sum()

Variable containing:
 2
[torch.cuda.ByteTensor of size 1 (GPU 0)]